### Exception Custom Classes with Logging Description and Solution

Suppose we have a Widget online sales application and we are writing the backend for it. We want a base `WidgetException` class that we will use as the base class for all our custom exceptions we raise from our Widget application.

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

Write an exception class hierarchy to capture this. In addition, we would like to implement the following functionality:
* implement separate internal error message and user error message
* implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request)
* implement a logging function that can be called to log the exception details, time it occurred, etc.
* implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)

In [83]:
from http import HTTPStatus
import json
import datetime
from traceback import TracebackException
import logging
logging.basicConfig(filename='exception_logs.log', encoding='utf-8', level = logging.DEBUG)

In [105]:
class WidgetException(Exception):
    """Base widget exception"""
    internal_error_message = "Widget API exception occurred."
    user_error_message = "We are sorry. An unexpected error occurred on our end."
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    def __init__(self, *args, user_error_message=None):
        if args:
            self.internal_error_message = args[0] #args[0] passed as custom exception message
            super().__init__(*args) 
        else:
            super().__init__(self.internal_error_message)
        if user_error_message is not None:
            self.user_error_message = user_error_message
        self.exception_time = datetime.datetime.utcnow()

    def exception_logger(self):
        """Logging function for logging the exception details"""
        log_message = (
            f"Exception type:{type(self).__name__}"+
            f"\nError message: {self.internal_error_message}\nHttp status: {HTTPStatus.INTERNAL_SERVER_ERROR}"+
            f"\nTime: {self.exception_time}\nArgs: {self.args}"+
            f'\nTBLogger: {"-".join(list((TracebackException.from_exception(self)).format()))}')
        logging.debug(log_message)
        return log_message
        
        
    def json_user_detail(self):
        """Returns a json string containing the exception details to display to the user"""
        error_object = {"Exception type":type(self).__name__,
                        'status':self.http_status, 
                        'message':self.user_error_message,
                         'time': str(self.exception_time)}
        return json.dumps(error_object)
     
    
class SupplierException(WidgetException):
    """Base supplier exception, inherits from WidgeException and indicates 
    that the exception occurred due to supply issues"""
    internal_error_message = "Supplier exception occurred."
    user_error_message = "We are sorry, a supplier exception occurred"

class CheckoutException(WidgetException):
    """Base checkout exception, inherits from WidgeException and indicates 
    that the exception occurred due to checkout issues"""
    internal_error_message = "Checkout exception occurred."
    user_error_message = "We are sorry, a checkout exception occurred"

# Supplier Exceptions
class NotManufacturedAnymoreException(SupplierException):
    """A SupplierException indicating that the item is no longer manufactured"""
    internal_error_message = "Manufacturing ceased exception occurred."
    user_error_message = "We are sorry, this product is no longer being made"

class ProductionDelayedException(SupplierException):
    """A SupplierException indicating that production is delayed"""
    internal_error_message = "Production delay exception occurred."
    user_error_message = "We are sorry, production was delayed"


class ShippingDelayedException(SupplierException):
    """A SupplierException indicating that shipping is delayed"""
    internal_error_message = "Shipping delay exception occurred."
    user_error_message = "We are sorry, shipping was delayed"


class OutOfStockException(CheckoutException):
    """A CheckoutException indicating that the item is out of stock"""
    internal_error_message = "Out of stock exception occurred."
    user_error_message = "We are sorry, item is out of stock"

### Pricing Exceptions
class PricingException(CheckoutException):
    http_status = HTTPStatus.BAD_REQUEST
    """Base pricing exception, inherits from CheckoutException and indicates 
    that the exception occurred due to pricing issues"""
    internal_error_message = "Base pricing exception occurred."
    user_error_message = "We are sorry, there was a problem with pricing"


class InvalidCouponCodeException(PricingException):
    """A PricingException indicating that the coupon code is invalid"""
    internal_error_message = "Invalid coupon code exception occurred."
    user_error_message = "We are sorry, this coupon code is invalid"

class CannotStackCouponsException(PricingException):
    """A PricingException indicating that coupons cannot be stacked"""
    internal_error_message = "Coupon stack exception occurred."
    user_error_message = "We are sorry, coupons cannot be stacked"


In [107]:
ex = WidgetException("hello exception")
print(ex.exception_logger())

Exception type:WidgetException
Error message: hello exception
Http status: 500
Time: 2022-05-10 13:40:13.479627
Args: ('hello exception',)
TBLogger: WidgetException: hello exception



In [109]:
#checking if the time works correctly (remains as exception creation time)
print(ex.exception_logger()) 

Exception type:WidgetException
Error message: hello exception
Http status: 500
Time: 2022-05-10 13:40:13.479627
Args: ('hello exception',)
TBLogger: WidgetException: hello exception



In [90]:
#Basic check of exception_logger traceback
try:
    raise WidgetException("hello exception")
except WidgetException as ex:
    print(ex.exception_logger())

Error message: hello exception
Http status: 500
Time: 2022-05-10 12:46:38.703825
Args: ('hello exception',)
TBLogger: Traceback (most recent call last):
-  File "<ipython-input-90-687a5dab50ec>", line 2, in <module>
    raise WidgetException("hello exception")
-WidgetException: hello exception



In [91]:
#Checking if can access http_status from subclass
ex = OutOfStockException() 
ex.json_user_detail()
print(ex.http_status)

HTTPStatus.INTERNAL_SERVER_ERROR


In [92]:
#Checking if methods work correctly in subclasses
ex = CannotStackCouponsException()
print(ex.json_user_detail())
print(ex.http_status)
ex.exception_logger()

{"status": 400, "message": "We are sorry, coupons cannot be stacked"}
HTTPStatus.BAD_REQUEST


"Error message: Coupon stack exception occurred.\nHttp status: 500\nTime: 2022-05-10 12:47:26.621761\nArgs: ('Coupon stack exception occurred.',)\nTBLogger: CannotStackCouponsException: Coupon stack exception occurred.\n"

In [106]:
#Checking json_user_detail
ex = InvalidCouponCodeException("Fast one pulled by the user","Invalid coupon, dear user")
ex.exception_logger()
ex.json_user_detail()

'{"Exception type": "InvalidCouponCodeException", "status": 400, "message": "We are sorry, this coupon code is invalid", "time": "2022-05-10 13:37:31.939060"}'